In [ ]:
import ifcopenshell.geom
import numpy

s = ifcopenshell.geom.settings(
    # USE_PYTHON_OPENCASCADE=True,
    USE_WORLD_COORDS=True,
    DISABLE_OPENING_SUBTRACTIONS=True)

f = ifcopenshell.open("models/Duplex_A_20110907_optimized.ifc")
it = ifcopenshell.geom.iterator(s, f, include=("IfcWall",))
t = ifcopenshell.geom.tree()
zs = {}
if it.initialize():
    for elem in ifcopenshell.geom.consume_iterator(it):
        t.add_element(it.get_native())
        arr = numpy.array(it.get().geometry.verts).reshape((-1, 3))
        zs[elem.id] = numpy.average(arr.T[2])
        

In [ ]:
import networkx as nx
import ifcopenshell.util.element
import bisect

storeys = sorted(s.Elevation for s in f.by_type('IfcBuildingStorey'))

g = nx.Graph()

def same_storey(a, b):
    s1, s2 = (ifcopenshell.util.element.get_container(x) for x in (a,b))
    return s1 == s2

def same_storey(a, b):
    s1, s2 = (bisect.bisect_left(storeys, zs[x]) for x in (a,b))
    return s1 == s2

for w in f.by_type("IfcWall"):
    for xid in [x.id() for x in t.select(w, extend=0.01)]:
        if xid != w.id():
            g.add_edge(w.id(), xid)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

N = nx.number_connected_components(g)
fig, axes = plt.subplots(1, N, figsize=(5*N, 5))
for i, comp in enumerate(nx.connected_components(g)):
    gg = nx.subgraph(g, comp)
    pos = nx.spring_layout(gg, iterations=1000)
    nx.draw(gg, pos=pos, ax=axes[i], node_size=10)
    

## Questions

* Why is this graph such a mess? (i.e not planar)
* What are the two connected components of 4 elements?
* How can we extend this to incorporate IfcSpace?